***
*Project:* Deep Active Inference

*Author:* Jingwei Liu (Music department, UC San Diego)
***

# <span style="background-color:darkorange; color:white; padding:2px 6px">Tutorial</span> 


# Version 3: DJ - Dancer Model (Guess action via limited past observations)

* Active niche construction

* With one sample delay

* Guess action via limited past observations

* Minimize under KL divergence

In [1]:
import numpy as np
import pandas as pd
import csv
from scamp import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Softmax,Input
import keras.backend as K 

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Species.csv', sep=",",index_col = 0)
n = df.shape[0]
X = tf.Variable(np.zeros((2,n*2)),dtype='int32')  # any data tensor
for i in range(0,2*n,2):
    X[0,i].assign(df['Cantus'][i/2] - 43)
    X[1,i].assign(0)
    X[0,i+1].assign(df['Counter'][i/2] - 55)
    X[1,i+1].assign(1)
X

<tf.Variable 'Variable:0' shape=(2, 20000) dtype=int32, numpy=
array([[15, 12, 23, ..., 10, 13,  9],
       [ 0,  1,  0, ...,  1,  0,  1]])>

In [3]:
depth = 30
X = tf.concat([tf.one_hot(X[0,:], depth),tf.dtypes.cast(tf.reshape(X[1,:],[20000,1]),tf.float32)],1)
X

<tf.Tensor: shape=(20000, 31), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices(X[:18000,:])
test_dataset = tf.data.Dataset.from_tensor_slices(X[18000:,:])

In [5]:
batch_size = 18

In [6]:
n_steps = 20
window_length = n_steps
dataset = test_dataset.window(window_length, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
shuffle_dataset = dataset.shuffle(20000).batch(batch_size,drop_remainder=True)

In [7]:
def creat_model(batch_size,Type):
    
    rnn_units = 12
    n_notes = 30

    input_A = keras.layers.Input(shape=(None,31), name="input_A") # (15, 20, 31)
    input_B = keras.layers.Input(shape=(1,31), name="input_B") # (15, 1, 31)

    hidden1 = keras.layers.LSTM(rnn_units, return_sequences=True)(input_A) # (15,20,12)
    hidden2 = keras.layers.LSTM(rnn_units, return_sequences=True)(hidden1) #(15,20,12)
    
    output_RNN = keras.layers.Dense(n_notes, activation='softmax', name = 'output_RNN')(hidden2) #(15,20,30)

    e = keras.layers.Dense(1, activation='tanh')(hidden2) #(15,20,1)
    e = keras.layers.Reshape([-1])(e) #(15,20)

    alpha = keras.layers.Activation('softmax')(e) #(15,20)
    c = keras.layers.Permute([2, 1])(keras.layers.RepeatVector(rnn_units)(alpha)) #(15,20,12)
    c = keras.layers.Multiply()([hidden2, c]) #(15,20,12)
    c = keras.layers.Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c) #(15,12)

    output_A = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_A')(c) #(15,30) (0, cantus)
    reshape = tf.reshape(output_A,[-1,1,30]) # (15, 1, 30)
    if Type == "_0_1":
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        input_output_A = tf.concat((reshape,zero),2) # (15, 1, 31)  (0, cantus)
    elif Type == "_1_0":
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        input_output_A = tf.concat((reshape,one),2) # (15, 1, 31)  (0, cantus)

    aux1 = keras.layers.SimpleRNN(rnn_units)(input_B, initial_state=c) #(15,12)
    aux2 = keras.layers.SimpleRNN(rnn_units)(input_output_A, initial_state=c) #(15,12)
    output_B1 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B1')(aux1) #(15,30) (1, counter)
    output_B2 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B2')(aux2) #(15,30) (1, counter)

    model = keras.models.Model([input_A, input_B], [output_RNN, output_A, output_B1, output_B2])
    
    aux_model1 = keras.models.Model([input_A, input_B], output_B1)
    aux_model2 = keras.models.Model(input_A, [output_A, output_B2])
    
    sub_model1 = keras.models.Model(input_A, output_A)
    sub_model2 = keras.models.Model(input_A, output_B2)
    
    att_model = keras.models.Model([input_A, input_B], alpha)
    
    return model,aux_model1,aux_model2,sub_model1,sub_model2,att_model

In [8]:
model_0_1_,aux_model1_0_1_,aux_model2_0_1_, sub_model1_0_1_,sub_model2_0_1_,att_model_0_1_ = creat_model(batch_size,'_0_1')
model_1_0_,aux_model1_1_0_,aux_model2_1_0_, sub_model1_1_0_,sub_model2_1_0_,att_model_1_0_ = creat_model(batch_size,'_1_0')

In [32]:
model_0_1_.load_weights('version3_model_0_1_step1_train1.h5')
model_1_0_.load_weights('version3_model_1_0_step1_train1.h5')

In [34]:
model_0_1_.load_weights('version3_model_0_1_step2_train1.h5')
model_1_0_.load_weights('version3_model_1_0_step2_train1.h5')

In [36]:
model_0_1_.load_weights('version3_model_0_1_step1_train1_random.h5')
model_1_0_.load_weights('version3_model_1_0_step1_train1_random.h5')

In [38]:
model_0_1_.load_weights('version3_model_0_1_step2_train1_random.h5')
model_1_0_.load_weights('version3_model_1_0_step2_train1_random.h5')

In [40]:
model_0_1_.load_weights('version3_model_0_1_step3_train1_random.h5')
model_1_0_.load_weights('version3_model_1_0_step3_train1_random.h5')

In [45]:
model_0_1_.load_weights('version3_model_0_1_step5_train1_random.h5')
model_1_0_.load_weights('version3_model_1_0_step5_train1_random.h5')

In [54]:
# version 3 generate sequence

seq_length = 10

#mode = "max"
mode = "random"

for window in shuffle_dataset.take(1):
    sequence = window
    initial = window
    
    X0 = np.zeros((batch_size,10,31))
    X1 = np.zeros((batch_size,10,31))
    
    weight_observation = np.zeros((batch_size,seq_length))
    action = np.zeros((batch_size,seq_length))
    weight_action_plus1 = np.zeros((batch_size,seq_length))
    observation = np.zeros((batch_size,seq_length))
    action_plus1 = np.zeros((batch_size,seq_length))
    
    for s in range(seq_length):
        for g in range(10):
            X0[:,g,:] = initial[:,2*g,:]
            X1[:,g,:] = initial[:,2*g+1,:]

        [pred_o,a] = aux_model2_0_1_.predict(X0) #(18,30)

        for i in range(batch_size):
            if mode == "max":
                action[i,s] = np.where(a[i,:] == np.max(a[i,:]))[0][0]
            elif mode == "random":
                action[i,s] = np.random.choice(30, 1, p=a[i,:])[0]

        one_hot = tf.one_hot(action[:,s], 30) #(18,30)

        reshape = tf.reshape(one_hot,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        input_action = tf.concat((reshape,one),2) # (18, 1, 31)  (1, counter)

        X1 = np.concatenate((X1[:,1:,:],input_action),axis=1)

        [pred_a_plus1,o] = aux_model2_1_0_.predict(X1)  
        #(18,30) # environment's prediction about t+1 action based on past actions; the observation the environment choose
       
        for i in range(batch_size):
            if mode == "max":                           
                observation[i,s] = np.where(o[i,:] == np.max(o[i,:]))[0][0]
            elif mode == "random":
                observation[i,s] = np.random.choice(30, 1, p=o[i,:])[0]
                
            index = int(observation[i,s])
            weight_observation[i,s] = pred_o[i,index]

        one_hot = tf.one_hot(observation[:,s], 30) #(18,30)

        reshape = tf.reshape(one_hot,[batch_size,1,30])
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        input_observation = tf.concat((reshape,zero),2) # (18, 1, 31)  (0, cantus)

        X0 = np.concatenate((X0[:,1:,:],input_observation),axis=1)

        [pred_o_plus1, a_plus1] = aux_model2_0_1_.predict(X0)

        for i in range(batch_size):
            if mode == "max":
                action_plus1[i,s] = np.where(a_plus1[i,:] == np.max(a_plus1[i,:]))[0][0]
            elif mode == "random":
                action_plus1[i,s] = np.random.choice(30, 1, p=a_plus1[i,:])[0]
                
            index = int(action_plus1[i,s])
            weight_action_plus1[i,s] = pred_a_plus1[i,index]

        sequence = tf.concat((sequence,input_observation,input_action),1)
        initial = sequence[:,-20:,:]
        
  
    print('action:',action)
    print('observation:',observation)
    print('action_plus1:',action_plus1)
    print('')
    
    
    step = [];
    for j in range(seq_length):
        step.append("step " + str(j))
    step.append("match#(sequence)")
        
    sample = [];
    for i in range(batch_size):
        sample.append("sequence " + str(i))
    sample.append("match#(step)")
    
    table = [];
    for i in range(batch_size):
        seq = [];
        for j in range(seq_length):
            seq.append(str(int(action[i,j])) + "/" + str(int(observation[i,j])) + "/(" + '%.2f'%(weight_observation[i,j]) + ")")
        match = np.sum(weight_observation[i,:])/seq_length
        seq.append('%.2f'%(match))
        table.append(seq)
        
    last_row = [];
    for j in range(seq_length):
        match = np.sum(weight_observation[:,j])/batch_size
        last_row.append('%.2f'%(match))
    total = np.sum(weight_observation)/(seq_length*batch_size)
    last_row.append('%.2f'%(total))
    table.append(last_row)
    
    format_row = "{:>13}" * (seq_length + 2)
    print(format_row.format("act/obv/(pred_obv_weight)", *step))
    for team, row in zip(sample, table):
        print(format_row.format(team, *row))
        
        
    table2 = [];
    for i in range(batch_size):
        seq = [];
        for j in range(seq_length):
            seq.append(str(int(observation[i,j]))  + "/" + str(int(action_plus1[i,j])) + "/(" + '%.2f'%(weight_action_plus1[i,j]) + ")")
        match = np.sum(weight_action_plus1[i,:])/seq_length
        seq.append('%.2f'%(match))
        table2.append(seq)
        
    last_row = [];
    total_num = 0;
    for j in range(seq_length):
        match = np.sum(weight_action_plus1[:,j])/batch_size
        last_row.append('%.2f'%(match))
    total = np.sum(weight_action_plus1)/(seq_length*batch_size)
    last_row.append('%.2f'%(total))
    table2.append(last_row)
    
    format_row = "{:>13}" * (seq_length + 2)
    print(format_row.format("obv/act1//(pred_act1_weight)", *step))
    for team, row in zip(sample, table2):
        print(format_row.format(team, *row))

action: [[10. 14. 24. 26. 17. 27. 28.  2.  0.  0.]
 [ 1. 16.  2. 17.  8. 16.  1. 25. 13. 16.]
 [28. 18.  0.  4. 23. 18. 16. 11. 12.  4.]
 [18.  9. 15.  0. 18.  0.  9. 15.  2.  1.]
 [11. 25. 11. 29.  5. 23. 10.  9. 14. 25.]
 [18. 11. 23.  5. 25. 25. 14.  4. 22. 28.]
 [ 0.  2. 11. 21. 10. 12. 11. 12. 19. 13.]
 [ 8. 15. 25. 27.  7. 25. 24.  5. 26. 28.]
 [ 6. 10. 10. 27.  8. 18.  6. 20. 12. 28.]
 [29. 27. 13.  7.  6.  9.  9. 19.  6.  9.]
 [21.  2. 27. 21.  1. 12. 28.  3. 27.  0.]
 [28. 25.  5. 12. 12. 26.  6. 13.  1.  3.]
 [ 8. 14.  6.  5. 14. 26. 14. 15.  1. 15.]
 [18. 24.  9.  1. 21.  9. 15. 28.  8. 24.]
 [13. 18.  1.  3. 10. 29. 13. 24. 20.  2.]
 [18.  3. 25. 25.  0. 14.  5.  5. 14. 12.]
 [ 0.  6. 25. 16. 28. 16. 12. 26.  0. 21.]
 [14.  8. 16. 25.  4.  3.  6. 14. 24. 21.]]
observation: [[23.  0. 19. 28. 20. 24.  0. 16. 22. 17.]
 [ 9.  9.  4.  4. 23.  9.  9.  3. 23. 12.]
 [18. 24.  8. 17. 19. 13. 20. 27.  4. 23.]
 [23. 29. 13.  8. 23. 10. 19.  8.  5. 23.]
 [27. 25. 22. 27. 14.  4.  7.  2

In [55]:
for sample in range(batch_size):
    music = tf.reshape(sequence[sample,:,:],[-1,31])
    #music = music[-20:,:]
    row,col = music.shape
    cantus_lst = [];
    counter_lst = [];
    for i in range(row):
        clas = np.where(music[i,:-1] == 1)[0][0]
        if music[i,-1] == 0:
            cantus_lst.append(clas+43)
        elif music[i,-1] == 1:
            counter_lst.append(clas+55)
    print(np.array(counter_lst) - np.array(cantus_lst))

[  3   8   9   4   9   8  12   8   4   7  -1  26  17  10   9  15  40  -2
 -10  -5]
[ 3  7  4  9  8  8  9  7  9  4  4 19 10 25 -3 19  4 34  2 16]
[ 9  8  9  9 12  3  8  8  3  4 22  6  4 -1 16 17  8 -4 20 -7]
[  3   8   9   9   8   8   4   4   9   3   7  -8  14   4   7   2   2  19
   9 -10]
[ 9  9  7  4 12  9  8  8 12  3 -4 12  1 14  3 31 15 19 -2 21]
[ 9  7  3  9  7  9  8  8  3  4 21 16 31 -7 24 24 16  7 32 35]
[ 8  3  7  4  9  8  8  9  7  9 -6 14 17  8  0  1  5 20  4  5]
[ 3  7  4  3  4  7 12  8  8  8  5 26 12 38 -3 24 34  3 11 39]
[ 4  8  9  9  9  3  8  4  4  8 16  2  4 15  0 24 10 24 -5 19]
[12  7 12  9  9  9  4 12  8  4 17 17 20 14 -1 -3 -8 25 13  8]
[ 8  4  3  4  3  8  9  4  9  8 19 -3 35 28 -4  1 17 14 19 -6]
[  8   3   4   4   3   3   8   9   4   9  22  36  14  -3  17  16  -9  24
  -3 -14]
[  7   8   4   3   7   3   4   8   9   9  15  17   8 -11   4  10  -2   9
  -6   2]
[ 3  8  8  8  4  7 12  9 12  9  5 33 13 -2 16  3 -1 27  0 23]
[ 4  9  8 12  4  8  8  4  3  7  5 18  4  1 11 28

In [56]:
sample = 12
music = tf.reshape(sequence[sample,:,:],[-1,31])
#music = music[-20:,:]
row,col = music.shape
cantus_lst = [];
counter_lst = [];
for i in range(row):
    clas = np.where(music[i,:-1] == 1)[0][0]
    if music[i,-1] == 0:
        cantus_lst.append(clas+43)
    elif music[i,-1] == 1:
        counter_lst.append(clas+55)

In [57]:
cantus_lst

[57,
 59,
 66,
 64,
 61,
 58,
 59,
 57,
 54,
 55,
 48,
 52,
 53,
 71,
 65,
 71,
 71,
 61,
 62,
 68]

In [58]:
counter_lst

[64,
 67,
 70,
 67,
 68,
 61,
 63,
 65,
 63,
 64,
 63,
 69,
 61,
 60,
 69,
 81,
 69,
 70,
 56,
 70]

In [59]:
np.array(counter_lst) - np.array(cantus_lst) # real harmonic

array([  7,   8,   4,   3,   7,   3,   4,   8,   9,   9,  15,  17,   8,
       -11,   4,  10,  -2,   9,  -6,   2], dtype=int64)

In [60]:
np.diff(counter_lst)

array([  3,   3,  -3,   1,  -7,   2,   2,  -2,   1,  -1,   6,  -8,  -1,
         9,  12, -12,   1, -14,  14], dtype=int64)

In [61]:
np.diff(cantus_lst)

array([  2,   7,  -2,  -3,  -3,   1,  -2,  -3,   1,  -7,   4,   1,  18,
        -6,   6,   0, -10,   1,   6], dtype=int64)

In [63]:
s = Session()
s.tempo = 200
piano1 = s.new_part("piano")
piano2 = s.new_part("piano")

def cantus():
    for i in cantus_lst:
        piano2.play_note(i,1,4)
        
def counter():
    for i in counter_lst:
        piano1.play_note(i,1,4)
        
        
s.fast_forward_to_beat(100)

s.start_transcribing()
s.fork(counter)
s.fork(cantus)
s.wait_for_children_to_finish()
performance = s.stop_transcribing()
performance.to_score(title = "First Species Counterpoint", composer = "My programme",time_signature = "4/4").show_xml()

Using preset Piano Merlin for piano
Using preset Piano Merlin for piano
